In [1]:
import warnings
warnings.filterwarnings('ignore')
from matplotlib import font_manager,rc
f_name=font_manager.FontProperties(fname='C:/Windows/Fonts/NGULIM.ttf').get_name()
rc('font',family=f_name)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from fbprophet import Prophet 

Importing plotly failed. Interactive plots will not work.


In [3]:
data=pd.read_csv("eco jeju/final_dataset.csv")

In [4]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [5]:
data=data.loc[:,:'제주_resd_pop']

In [6]:
data['year']=data['year'].astype(str)
data['month']=data['month'].astype(str)
data['date']=data['date'].astype(str)

In [7]:
data['base_date']=data['year']+"-"+data['month']+"-"+data['date']
data['base_date']=pd.to_datetime(data['base_date'])

In [8]:
data.drop(['year','month','date'],axis=1,inplace=True)

In [9]:
data

,emd_nm,em_g,total_pop,제주_resd_ratio,제주_resd_pop,base_date
0,건입동,1708250,10028,0.552623,7274.890502,2018-01-01
1,남원읍,1239600,19740,0.704247,12143.821692,2018-01-01
2,노형동,9357900,54958,0.769356,50660.687565,2018-01-01
3,대륜동,1717700,13651,0.667224,11185.575639,2018-01-01
4,대정읍,1264950,22603,0.784751,16075.181182,2018-01-01
...,...,...,...,...,...,...
50881,한경면,973850,9495,0.749118,7257.895822,2021-06-30
50882,한림읍,3298100,24462,0.769060,20632.727912,2021-06-30
50883,화북동,3749600,24630,0.705806,24117.268821,2021-06-30
50884,효돈동,1011050,5314,0.744489,3726.154500,2021-06-30


In [10]:
##2021년 5월 30일 train, 6월 1일에서 6월 18일까지 test로
data_train=data.loc[data['base_date']<='2021-05-30']

In [11]:
data_test=data.loc[(data['base_date']>='2021-06-01')&(data['base_date']<='2021-06-18')]

In [12]:
data_test

,emd_nm,em_g,total_pop,제주_resd_ratio,제주_resd_pop,base_date
49638,건입동,1320000,9110,0.536131,6749.072768,2021-06-01
49639,구좌읍,1179950,15955,0.762469,13881.063572,2021-06-01
49640,남원읍,1770800,19354,0.760316,14736.993588,2021-06-01
49641,노형동,7501050,57473,0.727099,54738.205330,2021-06-01
49642,대륜동,1494100,15277,0.724267,14593.421729,2021-06-01
...,...,...,...,...,...,...
50389,한경면,884350,9495,0.754707,7363.618917,2021-06-18
50390,한림읍,2575100,24462,0.775363,20990.875850,2021-06-18
50391,화북동,3273750,24630,0.711459,24719.499267,2021-06-18
50392,효돈동,929950,5314,0.760800,3856.153048,2021-06-18


# total_pop예측

In [51]:
test_df=pd.DataFrame(columns=['emd_nm','base_date','total_pop'])

In [52]:
for region in data['emd_nm'].unique():
    data_pop=data_train.loc[data_train['emd_nm']==region,['total_pop','base_date']]
    data_pop.rename(columns={'total_pop':'y','base_date':'ds'},inplace=True)
    model=Prophet(seasonality_mode = 'multiplicative',daily_seasonality=True,yearly_seasonality=True,weekly_seasonality=True)
    model.fit(data_pop)
    future=model.make_future_dataframe(periods=19,freq='D')
    forecast=model.predict(future)
    forecast=forecast[['ds','yhat']]
    forecast.rename(columns={'ds':'base_date','yhat':'total_pop'},inplace=True)
    forecast['emd_nm']=region
    test_df=pd.concat([test_df,forecast])

In [53]:
test_df=test_df.loc[(test_df['base_date']>='2021-06-01')&(test_df['base_date']<='2021-06-18')].sort_values(by=['base_date'])

In [54]:
test_df.rename(columns={'total_pop':'total_pop_pred'},inplace=True)

In [55]:
test_df

,emd_nm,base_date,total_pop_pred
1246,건입동,2021-06-01,9131.279307
575,조천읍,2021-06-01,25944.071073
1246,표선면,2021-06-01,12675.010161
1246,예래동,2021-06-01,4184.228124
1247,오라동,2021-06-01,16017.189014
...,...,...,...
1263,용담1동,2021-06-18,7021.182451
1264,외도동,2021-06-18,22361.672022
1264,오라동,2021-06-18,16040.454842
1263,일도1동,2021-06-18,2526.848229


In [56]:
data_test_total_pop=data_test[['base_date','emd_nm','total_pop']]

In [57]:
data_test_total_pop

,base_date,emd_nm,total_pop
49638,2021-06-01,건입동,9110
49639,2021-06-01,구좌읍,15955
49640,2021-06-01,남원읍,19354
49641,2021-06-01,노형동,57473
49642,2021-06-01,대륜동,15277
...,...,...,...
50389,2021-06-18,한경면,9495
50390,2021-06-18,한림읍,24462
50391,2021-06-18,화북동,24630
50392,2021-06-18,효돈동,5314


In [58]:
merge1=pd.merge(data_test_total_pop,test_df,on=['base_date','emd_nm'])

In [59]:
merge1

,base_date,emd_nm,total_pop,total_pop_pred
0,2021-06-01,건입동,9110,9131.279307
1,2021-06-01,구좌읍,15955,15966.495077
2,2021-06-01,남원읍,19354,19352.455751
3,2021-06-01,노형동,57473,57399.537681
4,2021-06-01,대륜동,15277,15249.793009
...,...,...,...,...
751,2021-06-18,한경면,9495,9434.076535
752,2021-06-18,한림읍,24462,24495.218479
753,2021-06-18,화북동,24630,24574.653660
754,2021-06-18,효돈동,5314,5329.750947


In [60]:
merge1['diff']=merge1['total_pop']-merge1['total_pop_pred']

In [61]:
merge1

,base_date,emd_nm,total_pop,total_pop_pred,diff
0,2021-06-01,건입동,9110,9131.279307,-21.279307
1,2021-06-01,구좌읍,15955,15966.495077,-11.495077
2,2021-06-01,남원읍,19354,19352.455751,1.544249
3,2021-06-01,노형동,57473,57399.537681,73.462319
4,2021-06-01,대륜동,15277,15249.793009,27.206991
...,...,...,...,...,...
751,2021-06-18,한경면,9495,9434.076535,60.923465
752,2021-06-18,한림읍,24462,24495.218479,-33.218479
753,2021-06-18,화북동,24630,24574.653660,55.346340
754,2021-06-18,효돈동,5314,5329.750947,-15.750947


In [62]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

print(mean_squared_error(merge1['total_pop'],merge1['total_pop_pred']))
print(mean_absolute_error(merge1['total_pop'],merge1['total_pop_pred']))

1751.1955566966103
30.02314492341151


In [ ]:
## changepoint_range=0.9,changepoint_prior_scale=0.8 / 2487.4366198240605
## changepoint_range=0.9,changepoint_prior_scale=0.5 / 2684.6628347376777
##낮출 수록 더 높아짐
## changepoint_range=0.9,changepoint_prior_scale=0.8 ,n_change=30 / 4595.806577137026
##  changepoint_range=0.9,changepoint_prior_scale=0.8 ,n_change=100 / 2444.7592496751813
##changepoint_range=0.9,changepoint_prior_scale=0.8,seasonality_mode = 'multiplicative',n_changepoints=100 / 1844.4889188087939
##seasonality_mode = 'multiplicative,daily_seansomality=True /1797.4913788113113
30.66051924039373
## changepoint_range=0.9,changepoint_prior_scale=0.8,seasonality_mode = 'multiplicative',n_changepoints=100,daily_seasonality=True / 2013.0073686585713
29.961670188926835
##seasonality_mode = 'multiplicative',daily_seasonality=True,yearly_seasonality=True,weekly_seasonality=True/1751.1955566966103
30.02314492341151

# 제주_ratio_pop 예측

In [74]:
test_ratio=pd.DataFrame(columns=['emd_nm','base_date','제주_resd_ratio'])

In [75]:
for region in data['emd_nm'].unique():
    data_ratio=data_train.loc[data_train['emd_nm']==region,['제주_resd_ratio','base_date']]
    data_ratio.rename(columns={'제주_resd_ratio':'y','base_date':'ds'},inplace=True)
    model=Prophet()
    model.fit(data_ratio)
    future=model.make_future_dataframe(periods=19,freq='D')
    forecast=model.predict(future)
    forecast=forecast[['ds','yhat']]
    forecast.rename(columns={'ds':'base_date','yhat':'제주_resd_ratio'},inplace=True)
    forecast['emd_nm']=region
    test_ratio=pd.concat([test_ratio,forecast])

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [76]:
test_ratio.loc[(test_ratio['base_date']>='2021-06-01')&(test_ratio['base_date']<='2021-06-18')].sort_values(by=['base_date'])

,emd_nm,base_date,제주_resd_ratio
1246,건입동,2021-06-01,0.545947
575,조천읍,2021-06-01,0.748520
1246,표선면,2021-06-01,0.783321
1246,예래동,2021-06-01,0.560939
1247,오라동,2021-06-01,0.535959
...,...,...,...
1263,용담1동,2021-06-18,0.608187
1264,외도동,2021-06-18,0.750080
1264,오라동,2021-06-18,0.536840
1263,일도1동,2021-06-18,0.370679


In [77]:
test_ratio.rename(columns={'제주_resd_ratio':'제주_resd_ratio_pred'},inplace=True)

In [78]:
data_test_ratio=data_test[['base_date','emd_nm','제주_resd_ratio']]

In [79]:
data_test_ratio

,base_date,emd_nm,제주_resd_ratio
49638,2021-06-01,건입동,0.536131
49639,2021-06-01,구좌읍,0.762469
49640,2021-06-01,남원읍,0.760316
49641,2021-06-01,노형동,0.727099
49642,2021-06-01,대륜동,0.724267
...,...,...,...
50389,2021-06-18,한경면,0.754707
50390,2021-06-18,한림읍,0.775363
50391,2021-06-18,화북동,0.711459
50392,2021-06-18,효돈동,0.760800


In [80]:
merge2=pd.merge(data_test_ratio,test_ratio,on=['base_date','emd_nm'])

In [81]:
merge2['diff']=merge2['제주_resd_ratio']-merge2['제주_resd_ratio_pred']

In [82]:
merge2

,base_date,emd_nm,제주_resd_ratio,제주_resd_ratio_pred,diff
0,2021-06-01,건입동,0.536131,0.545947,-0.009816
1,2021-06-01,구좌읍,0.762469,0.745361,0.017108
2,2021-06-01,남원읍,0.760316,0.760649,-0.000333
3,2021-06-01,노형동,0.727099,0.724394,0.002705
4,2021-06-01,대륜동,0.724267,0.709734,0.014533
...,...,...,...,...,...
751,2021-06-18,한경면,0.754707,0.730497,0.024210
752,2021-06-18,한림읍,0.775363,0.764293,0.011069
753,2021-06-18,화북동,0.711459,0.718409,-0.006950
754,2021-06-18,효돈동,0.760800,0.754946,0.005854


In [83]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

print(mean_squared_error(merge2['제주_resd_ratio'],merge2['제주_resd_ratio_pred']))
print(mean_absolute_error(merge2['제주_resd_ratio'],merge2['제주_resd_ratio_pred']))


0.0003613286397399271
0.013803496409631962


In [ ]:
##0.0003613286397399271 0.013803496409631962
## n_changepoints=200/ 0.0003662944889430445
##changepoint_range=0.9,changepoint_prior_scale=0.8,seasonality_mode = 'multiplicative' / 0.00036375842449593504
##changepoint_range=0.9,changepoint_prior_scale=0.8,seasonality_mode = 'multiplicative',n_changepoints=100/ 0.00038393730732293586
## changepoint_range=0.9,changepoint_prior_scale=0.5,seasonality_mode = 'multiplicative'/0.00038203231805676184
## seasonality_mode = 'multiplicative',daily_seasonality=True,yearly_seasonality=True,weekly_seasonality=True /0.0004273719476478369
0.014739895497804472

# 제주_resd_pop예측

In [129]:
test_resd_pop=pd.DataFrame(columns=['emd_nm','base_date','제주_resd_pop'])

In [130]:
for region in data['emd_nm'].unique():
    data_resd_pop=data_train.loc[data_train['emd_nm']==region,['제주_resd_pop','base_date']]
    data_resd_pop.rename(columns={'제주_resd_pop':'y','base_date':'ds'},inplace=True)
    model=Prophet(changepoint_range=0.5,changepoint_prior_scale=0.9,seasonality_mode = 'multiplicative',n_changepoints=200,daily_seasonality=True,yearly_seasonality=True,weekly_seasonality=True)
    model.fit(data_resd_pop)
    future=model.make_future_dataframe(periods=19,freq='D')
    forecast=model.predict(future)
    forecast=forecast[['ds','yhat']]
    forecast.rename(columns={'ds':'base_date','yhat':'제주_resd_pop'},inplace=True)
    forecast['emd_nm']=region
    test_resd_pop=pd.concat([test_resd_pop,forecast])

In [131]:
test_resd_pop.loc[(test_resd_pop['base_date']>='2021-06-01')&(test_resd_pop['base_date']<='2021-06-18')].sort_values(by=['base_date'])

,emd_nm,base_date,제주_resd_pop
1246,건입동,2021-06-01,6750.818446
575,조천읍,2021-06-01,19447.741125
1246,표선면,2021-06-01,10550.506703
1246,예래동,2021-06-01,3226.358631
1247,오라동,2021-06-01,12682.286344
...,...,...,...
1263,용담1동,2021-06-18,5880.013067
1264,외도동,2021-06-18,12459.915630
1264,오라동,2021-06-18,12310.728132
1263,일도1동,2021-06-18,1336.151678


In [132]:
test_resd_pop.rename(columns={'제주_resd_pop':'제주_resd_pop_pred'},inplace=True)

In [133]:
data_resd_pop=data_test[['base_date','emd_nm','제주_resd_pop']]

In [134]:
merge3=pd.merge(data_resd_pop,test_resd_pop,on=['base_date','emd_nm'])

In [135]:
merge3

,base_date,emd_nm,제주_resd_pop,제주_resd_pop_pred
0,2021-06-01,건입동,6749.072768,6750.818446
1,2021-06-01,구좌읍,13881.063572,12985.887284
2,2021-06-01,남원읍,14736.993588,14734.613066
3,2021-06-01,노형동,54738.205330,53018.963251
4,2021-06-01,대륜동,14593.421729,14019.474908
...,...,...,...,...
751,2021-06-18,한경면,7363.618917,6673.251721
752,2021-06-18,한림읍,20990.875850,20638.535689
753,2021-06-18,화북동,24719.499267,25656.838759
754,2021-06-18,효돈동,3856.153048,3833.123885


In [136]:
merge3['diff']=merge3['제주_resd_pop']-merge3['제주_resd_pop_pred']

In [137]:
merge3

,base_date,emd_nm,제주_resd_pop,제주_resd_pop_pred,diff
0,2021-06-01,건입동,6749.072768,6750.818446,-1.745679
1,2021-06-01,구좌읍,13881.063572,12985.887284,895.176288
2,2021-06-01,남원읍,14736.993588,14734.613066,2.380522
3,2021-06-01,노형동,54738.205330,53018.963251,1719.242079
4,2021-06-01,대륜동,14593.421729,14019.474908,573.946820
...,...,...,...,...,...
751,2021-06-18,한경면,7363.618917,6673.251721,690.367197
752,2021-06-18,한림읍,20990.875850,20638.535689,352.340161
753,2021-06-18,화북동,24719.499267,25656.838759,-937.339492
754,2021-06-18,효돈동,3856.153048,3833.123885,23.029163


In [138]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

print(mean_squared_error(merge3['제주_resd_pop'],merge3['제주_resd_pop_pred']))
print(mean_absolute_error(merge3['제주_resd_pop'],merge3['제주_resd_pop_pred']))

3363756.427981026
696.7012800511004


In [ ]:
## 2620984.964723936,488.7013213300222
## changepoint_range=0.9,changepoint_prior_scale=0.8,seasonality_mode = 'multiplicative' / 3110299.034999392
##changepoint_range=0.9,changepoint_prior_scale=0.5,seasonality_mode = 'multiplicative' /  3547613.8923417297
## changepoint_range=0.5,changepoint_prior_scale=0.9,seasonality_mode = 'multiplicative' /2711263.797436628
559.8363126580617
## changepoint_range=0.5,changepoint_prior_scale=0.9,seasonality_mode = 'multiplicative',n_changepoints=100 /2654163.450804129
558.8411336449889
## changepoint_range=0.5,changepoint_prior_scale=0.9,seasonality_mode = 'multiplicative',n_changepoints=200 /2649431.955449781
##seasonality_mode = 'multiplicative',daily_seasonality=True,yearly_seasonality=True,weekly_seasonality=True /4219230.948335799
597.2684645213172
##changepoint_range=0.5,changepoint_prior_scale=0.9,seasonality_mode = 'multiplicative',n_changepoints=200,daily_seasonality=True,yearly_seasonality=True,weekly_seasonality=True
##3363756.427981026
696.7012800511004

# 7,8월 데이터 예측